In [1]:
import io
import os
import tempfile

import requests
import pyarrow as pa
import pyarrow.parquet as pq

import tqdm.notebook as tqdm

In [2]:
gcp_credentials_path = '~/Workspace/DTC/DE/credentials/gcp.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.expanduser(gcp_credentials_path)

bucket_name = 'tsbalzhanov-dtc-week-3'
gcs_file_path = f'{bucket_name}/green_taxi_data/green_tripdata_2022'
gcs_fs = pa.fs.GcsFileSystem()

In [3]:
year = 2022

pa_tables = []

for month in tqdm.tnrange(1, 13):
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year}-{month:02}.parquet'
    file_name = url.split('/')[-1]

    result = requests.get(url)
    result.raise_for_status()

    parquet_file = io.BytesIO(result.content)
    pa_tables.append(pq.read_table(parquet_file))

  0%|          | 0/12 [00:00<?, ?it/s]

In [4]:
combined_table = pa.concat_tables(pa_tables)
pq.write_table(combined_table, gcs_file_path, filesystem=gcs_fs)